<a href="https://colab.research.google.com/github/josefernandezest98/training/blob/main/Fruit_Thresholding/fruit_threshold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fruit Thresholding